# **INFO5731 In-class Exercise 4**

**This exercise will provide a valuable learning experience in working with text data and extracting features using various topic modeling algorithms. Key concepts such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)

**Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [3]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 34.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


In [35]:
# Write your code here
import nltk
nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
# spacy for lemmatization
import spacy
from gensim.models import LdaModel
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
sample='This is a great little speaker! You get hours of play time and the sound is even better than I expected. The size is great, you can literally throw it in your pocket or clip it to a bag. It’s very easy to set up and use. I got it for when I go paddle boarding, but it works so well I use it all the time now.'


In [38]:
df = [i.strip() for i in re.split(r'(?<=[.!?])\s+', sample)]
print(df)


['This is a great little speaker!', 'You get hours of play time and the sound is even better than I expected.', 'The size is great, you can literally throw it in your pocket or clip it to a bag.', 'It’s very easy to set up and use.', 'I got it for when I go paddle boarding, but it works so well I use it all the time now.']


In [39]:
# Tokenize the text
df= [doc.split() for doc in df]

# Create a dictionary and document-term matrix
dictionary = Dictionary(df)
df = [dictionary.doc2bow(doc) for doc in df]


In [40]:
coherence_scores = []
for k in range(2, 10):
    lda_model = LdaModel(df, num_topics=k, id2word=dictionary, passes=15)
    coherence_model = CoherenceModel(model=lda_model, texts=df, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append((k, coherence_score))

# Find the K with the highest coherence score
optimal_k = max(coherence_scores, key=lambda x: x[1])[0]

# Train the final LDA model with the selected K
lda_model = LdaModel(df, num_topics=optimal_k, id2word=dictionary, passes=15)

# Print and summarize the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Print the selected K and its coherence score
print(f"Optimal K: {optimal_k}")

/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


(0, '0.043*"and" + 0.043*"to" + 0.043*"easy" + 0.043*"set" + 0.043*"up" + 0.043*"use." + 0.043*"It’s" + 0.043*"very" + 0.014*"great" + 0.014*"speaker!"')
(1, '0.063*"it" + 0.052*"I" + 0.040*"is" + 0.029*"time" + 0.029*"the" + 0.029*"a" + 0.017*"works" + 0.017*"boarding," + 0.017*"use" + 0.017*"for"')
Optimal K: 2


## Question 2 (10 Points)

**Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [41]:
# Write your code here
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import LsiModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel


In [43]:
sample='This is a great little speaker! You get hours of play time and the sound is even better than I expected. The size is great, you can literally throw it in your pocket or clip it to a bag. It’s very easy to set up and use. I got it for when I go paddle boarding, but it works so well I use it all the time now.'
df = [i.strip() for i in re.split(r'(?<=[.!?])\s+', sample)]
print(df)



['This is a great little speaker!', 'You get hours of play time and the sound is even better than I expected.', 'The size is great, you can literally throw it in your pocket or clip it to a bag.', 'It’s very easy to set up and use.', 'I got it for when I go paddle boarding, but it works so well I use it all the time now.']


In [44]:
# Tokenize the text
tokenized_text= [doc.split() for doc in df]

# Create a dictionary and document-term matrix
dictionary = Dictionary(tokenized_text)
corpus= [dictionary.doc2bow(doc) for doc in tokenized_text]


In [45]:

# Generate coherence scores for different numbers of topics (K)
def compute_coherence_values(dictionary, corpus, texts, limit):
    coherence_values = []
    for num_topics in range(2, limit + 1):
        # Build LSI model
        lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=num_topics)

        # Compute coherence score
        coherence_model = CoherenceModel(model=lsi_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())

    return coherence_values

# Set the upper limit for the number of topics (K)
limit = 10

# Compute coherence scores for different values of K
coherence_values = compute_coherence_values(dictionary, corpus, tokenized_text, limit)

# Find the optimal number of topics based on the highest coherence score
optimal_k = coherence_values.index(max(coherence_values)) + 2  # Adding 2 because the loop starts from 2
print(f"Optimal number of topics (K): {optimal_k}")

# Build the final LSA model with the optimal number of topics
final_lsa_model = LsiModel(corpus, id2word=dictionary, num_topics=optimal_k)

# Print the topics
topics = final_lsa_model.show_topics(num_words=10)
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")

Optimal number of topics (K): 3
Topic 0: -0.566*"it" + -0.489*"I" + -0.188*"the" + -0.188*"time" + -0.150*"paddle" + -0.150*"but" + -0.150*"all" + -0.150*"boarding," + -0.150*"use" + -0.150*"so"
Topic 1: -0.262*"I" + 0.249*"a" + 0.234*"to" + 0.216*"The" + 0.216*"bag." + 0.216*"can" + 0.216*"you" + 0.216*"or" + 0.216*"literally" + 0.216*"pocket"
Topic 2: 0.296*"and" + 0.294*"is" + 0.252*"You" + 0.252*"even" + 0.252*"play" + 0.252*"get" + 0.252*"hours" + 0.252*"of" + 0.252*"than" + 0.252*"better"


## Question 3 (10 points):
**Generate K topics by using lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [56]:
!pip install lda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.8/349.8 kB 7.8 MB/s eta 0:00:00


In [57]:
!pip install lda2vec

In [58]:
!pip install git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis

  Cloning https://github.com/bmabey/pyLDAvis.git (to revision master) to /tmp/pip-install-8gbiy7kc/pyldavis_b2c9bd82efa24e72b078a7b9e6648701
  Running command git clone --filter=blob:none --quiet https://github.com/bmabey/pyLDAvis.git /tmp/pip-install-8gbiy7kc/pyldavis_b2c9bd82efa24e72b078a7b9e6648701
  Resolved https://github.com/bmabey/pyLDAvis.git to commit 16800f36bc95b4c99d8c26d51daa3485c8cb76da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [59]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi

In [60]:
import pandas as pd
from lda2vec.model import LDA2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import matplotlib.pyplot as plt

sample='This is a great little speaker! You get hours of play time and the sound is even better than I expected. The size is great, you can literally throw it in your pocket or clip it to a bag. It’s very easy to set up and use. I got it for when I go paddle boarding, but it works so well I use it all the time now.'
df = [i.strip() for i in re.split(r'(?<=[.!?])\s+', sample)]

# Tokenize the text
tokenized_text= [doc.split() for doc in df]

# Create a dictionary and document-term matrix
dictionary = Dictionary(tokenized_text)
corpus= [dictionary.doc2bow(doc) for doc in tokenized_text]


""" Since lda2vec has importing issues , I thought working on the alternate 3rd Question"""

ModuleNotFoundError: No module named 'lda2vec.model'

## Question 4 (10 points):
**Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [78]:
!pip install bertopic

In [95]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score
import numpy as np

sample='This is a great little speaker! You get hours of play time and the sound is even better than I expected. The size is great, you can literally throw it in your pocket or clip it to a bag. It’s very easy to set up and use. I got it for when I go paddle boarding, but it works so well I use it all the time now.'
df = [i.strip() for i in re.split(r'(?<=[.!?])\s+', sample)]
print(df)

# Tokenize the text
tokenized_text= [doc.split() for doc in df]

# Create a dictionary and document-term matrix
dictionary = Dictionary(tokenized_text)
corpus= [dictionary.doc2bow(doc) for doc in tokenized_text]


['This is a great little speaker!', 'You get hours of play time and the sound is even better than I expected.', 'The size is great, you can literally throw it in your pocket or clip it to a bag.', 'It’s very easy to set up and use.', 'I got it for when I go paddle boarding, but it works so well I use it all the time now.']


In [93]:
# Create a bag-of-words matrix using CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
x = vectorizer.fit_transform(tokenized_text)

# Reduce dimensionality using Truncated SVD
svd_model = TruncatedSVD(n_components=5)
X_svd = svd_model.fit_transform(x)


AttributeError: 'list' object has no attribute 'lower'

In [100]:
# Write your code here
# The sample list of string data that which I considered is giving error While bert topic input itself , instead I want to try using different data

from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']


In [102]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

freq = topic_model.get_topic_info()
freq.head(5)
topic_model.get_topic(0) #this gives the most frequent words

2024-03-29 03:50:04,388 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2024-03-29 04:14:07,462 - BERTopic - Embedding - Completed ✓
2024-03-29 04:14:07,464 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-29 04:14:36,772 - BERTopic - Dimensionality - Completed ✓
2024-03-29 04:14:36,775 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-29 04:15:12,756 - BERTopic - Cluster - Completed ✓
2024-03-29 04:15:12,767 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-29 04:15:18,498 - BERTopic - Representation - Completed ✓


[('game', 0.010506260225635314),
 ('team', 0.009140167990825862),
 ('games', 0.007276825659579582),
 ('he', 0.0072115120960511685),
 ('players', 0.006395973392086214),
 ('season', 0.0063136201945379715),
 ('hockey', 0.006187511633454982),
 ('play', 0.005849442846785478),
 ('25', 0.005723942703849343),
 ('year', 0.005695628761819004)]

## **Question 3 (Alternative) - (10 points)**

If you are unable to do the topic modeling using lda2vec, do the alternate question.

Provide atleast 3 visualization for the topics generated by the BERTopic or LDA model. Explain each of the visualization in detail.

In [ ]:
!pip install bertopic umap-learn matplotlib

In [107]:
# from the above 4th question data and BERTopic model, we can get the visualisation plots.
topic_model.visualize_topics()

In [105]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [106]:
topic_model.visualize_barchart(top_n_topics=5)

## Extra Question (5 Points)

**Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.**

**This question will compensate for any points deducted in this exercise. Maximum marks for the exercise is 40 points.**

In [ ]:
# Write your code here
"""
Because topic models cannot ensure that their output is interpretable, Topic Coherence can be used to assess them. Topic modeling enables us to cut through the noise (the high dimensionality of text data) and notice the signal (the main topics) in our data.

There are various Topic Modeling methodologies, the most popular of which being Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation.
LDA is the most feasible and extensively used approach for designing and implementing Topic Modeling.

LDA2vec is a derivative of LDA that incorporates word embeddings, which are dense, low-dimensional vector representations of words obtained from large text corpora using methods like Word2Vec. lda2vec uses each word as a vector in the very same space as the topics, allowing for advanced word-topic interactions.

BerTopic is a topic modeling technique that creates dense clusters using transformers (BERT embeddings) and class-based TF-IDF. It also enables you to easily interpret and visualize the generated topics.

The optimal value generated by LDA algorithm and LSA algorithm is 3. Both the algorithms have assessed the corpus and resulted the optimal values. As of now, lda2vec model import line of code isn't working even after going through multiple references online.

"""


# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.

Consider the following points in your response:

**Learning Experience:** Describe your overall learning experience in working with text data and extracting features using various topic modeling algorithms. Did you understand these algorithms and did the implementations helped in grasping the nuances of feature extraction from text data.

**Challenges Encountered:** Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

This assignment is a decent learning experience on learning the topic modelling algorithms. The implementation of these algorithms on feature extraction is good but the
4 assigment questions  with a time contraint is tough. Could've given some more time because the course project work is running on the parallel track.
That is the challenge that I encountered.

This is obviously part of NLP and text summarization. This topic modelling concept is really interesting. Very soon I am going to implement this concept in my 5731 project.


'''